
## Resumo do notebook

Neste notebook fazemos a importação de dados de uma fonte temporária do Databricks. 

Os principais passos incluem:

- Carregamento e visualização dos dados utilizando Spark.
- Utilização de funções para manipulação e processamento dos dados.
- Ingestão dos dados em uma tabela Delta para análises futuras.
- Inclusão de informações na tabela.

In [0]:
from pyspark.sql.functions import upper, trim, col, try_to_timestamp

cap_path = "/Volumes/ons/staging/capacidade_geracao"
cap_df = spark.read.format("parquet").load(f"{cap_path}/*.parquet")

cap_df = cap_df.withColumn('id_subsistema', upper(trim(col('id_subsistema'))))
cap_df = cap_df.withColumn('nom_subsistema', upper(trim(col('nom_subsistema'))))

cap_df = cap_df.withColumn('dat_entradateste', try_to_timestamp(col('dat_entradateste')))
cap_df = cap_df.withColumn('dat_entradaoperacao', try_to_timestamp(col('dat_entradaoperacao')))
cap_df = cap_df.withColumn('dat_desativacao', try_to_timestamp(col('dat_desativacao')))

cap_df = cap_df.withColumn('val_potenciaefetiva', col('val_potenciaefetiva').cast('double'))

# cap_df.printSchema()

cap_df.write.format("delta").mode("overwrite").saveAsTable("ons.dados.capacidade_geracao")

display(cap_df.limit(10))

In [0]:
spark.sql("""
COMMENT ON TABLE ons.dados.capacidade_geracao IS
'Tabela contém informações sobre subsistemas operacionais e seus status. Inclui detalhes como nomes de subsistemas, estados operacionais e agentes associados. Esses dados podem ser utilizados para monitorar o desempenho dos diferentes subsistemas, analisar a eficiência operacional e acompanhar mudanças ao longo do tempo, incluindo datas de ativação e desativação.'
""")

In [0]:
desc_cap = {
    "id_subsistema": "Código do Subsistema da Usina",
    "nom_subsistema": "Nome do Subsistema",
    "id_estado": "Sigla do Estado onde está a Usina",
    "nom_estado": "Nome do Estado",
    "nom_modalidadeoperacao": "Modalidade de Operação da Usina",
    "nom_agenteproprietario": "Agente Proprietário da Usina",
    "nom_agenteoperador": "Agente Operador da Usina",
    "nom_tipousina": "Tipo da Usina",
    "nom_usina": "Nome da Usina",
    "ceg": "Código Único do Empreendimento de Geração (CEG)",
    "nom_unidadegeradora": "Nome da Unidade Geradora",
    "cod_equipamento": "Código do Equipamento – Unidade Geradora",
    "num_unidadegeradora": "Código Operacional da Unidade Geradora",
    "nom_combustivel": "Combustível da Unidade Geradora",
    "dat_entradateste": "Data de Entrada em Teste (Comissionamento)",
    "dat_entradaoperacao": "Data de Entrada em Operação Comercial",
    "dat_desativacao": "Data de Desativação da Unidade Geradora",
    "val_potenciaefetiva": "Potência Nominal da Unidade Geradora, em MW"
}

for col_name, col_desc in desc_cap.items():
    spark.sql(f"COMMENT ON COLUMN ons.dados.capacidade_geracao.{col_name} IS '{col_desc}'")